In [ ]:
import tensorflow as tf

## CNN

### 1. 自定义权值

In [ ]:
x = tf.random.normal([2, 5, 5, 3]) # 两张5*5的图片，RGB三个通道
w = tf.random.normal([3, 3, 3, 4]) # 4个3*3且3个通道的卷积核

# 进行卷积计算，步长位1，padding为0
out = tf.nn.conv2d(x, w, strides = 1, padding = [[0,0],[0,0],[0,0],[0,0]])
out.shape

上面得到的out的形状，第一个2代表两张图片，第一个和第二个3代表得到经过卷积核运算后得到的矩阵大小，最后一个4是因为一共有4个卷积核。

In [ ]:
# 我们让每张图片的上下左右都填充一个步长
out = tf.nn.conv2d(x, w, strides = 1, padding = [[0,0],[1,1],[1,1],[0,0]])
out.shape

In [ ]:
# 我们也可以让padding = same，这样tensorflow自动运算完成填充
out = tf.nn.conv2d(x, w, strides = 1, padding = 'SAME')
out.shape

In [ ]:
# 对于一张变化不是很大的图片，步长可以设置的大一些
out = tf.nn.conv2d(x, w, strides = 3, padding = 'SAME')
out.shape

In [ ]:
# 如果需要偏置则需要手动添加偏置
b = tf.zeros([4])
out = out + b # 这里自动使用广播法则

### 2. 卷积层类

In [ ]:
layer = tf.keras.layers.Conv2D(4, # 卷积核的数量 
                               kernel_size = 3,  # 卷积核的形状为3*3
                               strides = 1,  # 步长
                               padding = 'SAME' # 填充
                              )
out = layer(x)
out.shape

可以查看待优化张量列表，包括权值矩阵W和偏置b

In [ ]:
# 查看待优化参数
# layer.trainable_variables
# layer.kernel
# layer.bias

### 3. 卷积神经网络示例

In [ ]:
from tensorflow.keras import Sequential, layers

In [ ]:
CNN = Sequential([
    layers.Conv2D(6, kernel_size = 3, strides = 1, padding = 'SAME'), # 卷积层
    layers.MaxPooling2D(pool_size = 2, strides = 2), # 池化层
    layers.ReLU(), # 激活函数
    layers.Conv2D(16, kernel_size = 3, strides = 1),
    layers.MaxPooling2D(pool_size = 2, strides = 2),
    layers.ReLU(),
    layers.Flatten(), # 打平层，方便全连接处理
    layers.Dense(120, activation = 'relu'),
    layers.Dense(60, activation = 'relu'),
    layers.Dense(10)
])
CNN.build(input_shape = (4, 28, 28, 1))
CNN.summary()

In [ ]:
from tensorflow.keras import losses, optimizers
# 使用mnist数据集
criteon = losses.CategoricalCrossentropy(from_logits = True)
with tf.GradientTape() as tape:
    x = tf.expand_dims(x, axis = 3)
    out = CNN(x)
    y_onehot = tf.one_hot(y, depth = 10)
    loss = criteon(y_onehot, out)
grads = tape.gradient(loss, CNN.trainable_variables)
optimizers.apply_gradients(zip(grads, CNN.trainable_variables))

### 4. 表示学习
通过将每层的特征图利用反卷积网络(DeconvolutionalNetwork)映射回输入图片，即可查看学到的特征分布，可以观察到，第二层的特征对应到边、角、色彩等底层图像提取；第三层开始捕获到纹理这些中层特征；第四、五层呈现了物体的部分特征，如小狗的脸部，鸟类的脚部等高层特征。 通过这些可视化的手段，我们可以一定程度上感受卷积神经网络的特征学习过程。

图片数据的识别过程一般认为也是表示学习(Representation Learning)的过程，从接受到的原始像素开始，逐渐提取边缘、角点等底层特征，再到纹理等中层特征，再到头部、物体部件等高层特征，最后的网络层基于这些学习到的抽象特征表示(Representation)做分类逻辑的学习。学习到的特征越高层，越准确，就越有利于分类器的分类，从而获得较好的性能。从表示学习的角度来理解，卷积神经网络通过层层堆叠来逐层提取特征，网络训练的过程可以看成特征的学习过程，基于学习到的高层抽象特征可以方便地进行分类任务。

应用表示学习的思想，训练好的卷积神经网络往往能够学习到较好的特征，这种特征的提取方法一般是通用的。比如在猫、 狗任务上学习到头、脚、身躯等特征的表示，在其他动物上也能够一定程度上使用。基于这种思想，可以将在任务 A 上训练好的深层神经网络的前面数个特征提取层迁移到任务 B 上，只需要训练任务 B 的分类逻辑(表现为网络的最末数层)，即可取得非常好的效果，这种方式是迁移学习的一种，从神经网络角度也称为网络微调。

### 5. CNN网络的梯度传播
通过循环移动感受野的方式并没有改变网络层可导性，同时梯度的推导也并不复杂，只是当网络层数增大以后，人工梯度推导将变得十分的繁琐。

### 6. 池化层
池化层同样基于局部相关性的思想，通过从局部相关的一组元素中进行采样或信息聚合，从而得到新的元素值。特别地，最大池化层(Max Pooling)从局部相关元素集中选取最大的一个元素值，平均池化层(Average Pooling)从局部相关元素集中计算平均值并返回。

由于池化层没有需要学习的参数，计算简单，可以有效减低特征图的尺寸，非常适合图片这种类型的数据。

### 7. BatchNorm层
卷积神经网络的出现，网络参数量大大减低，使得几十层的深层网络成为可能。然而，在残差网络出现之前，网络的加深使得网络训练变得非常不稳定， 甚至出现网络长时间不更新或者不收敛的情形，同时网络对超参数比较敏感，超参数的微量扰动也会导致网络的训练轨迹完全改变。

BN层是一种参数标准化的手段，它能让收敛更快，参数设定更加自由灵活。

我们使用Sigmoid函数时，为了避免梯度弥散，经常需要将输入参数进行标准化，标准化将值映射在0附近，此处的导数不至于过小，因此不容易出现梯度弥散的现象。

我们能够经验性归纳出：网络层输入𝑥分布相近， 并且分布在较小范围内时(如 0 附近)，更有利于函数的优化。 

标准化运算不引入额外的参数，因为均值和标准差都可以通过统计得到。

为了提高BN的表达能力，又引入了scale and shift技巧，即将标准化后的输入再次进行缩放和平移，但是此时控制缩放和平移的参数是需要经过反向传播算法进行自动优化的，这样可以实现网络层按需缩放和平移数据分布的目的。

而标准化也分为几种：
- Batch Norm
- Layer Norm：统计每个样本的所有特征的均值和方差
- Instance Norm：统计每个样本的每个通道上特征的均值和方差
- Group Norm：将c通道分成若干组，统计每个样本的通道组内的特征均值和方差

In [ ]:
# 创建BN层
layer = layers.BatchNormalization()

与全连接层、 卷积层不同， BN 层的训练阶段和测试阶段的行为不同，需要通过设置training 标志位来区分训练模式还是测试模式

In [ ]:
# # 在卷积层后添加BN层
# network = Sequential([
#     layers.Conv2D(6, kernel_size = 3, strides = 1),
#     # 插入BN层
#     layers.BatchNormalization(),
#     layers.MaxPooling2D(pool_size = 2, strides = 2)
#     # ...
# ])

In [ ]:
# # 在训练阶段，需要设置网络的参数training = True以区分BN层是训练还是测试模型
# with tf.GradientTape() as tape:
#     x = tf.expand_dims(x, axis = 3)
#     out = network(x, training = True) # 设置training = True，区分BN层是训练还是测试模型

# # 在测试阶段，设置training = False，避免BN层采用错误的行为：
# for x, y in db_test:
#     x = tf.expand_dims(x, axis = 3)
#     out = network(x, training = False)

### 8. 经典CNN
#### AlexNet
- 层数达到8层
- 采用了ReLU激活函数
- 引入了Dropout

#### VGG系列
- 层数提升到19层
- 采用更小的3\*3卷积核，AlexNet卷积核为7\*7
- 采用更小的池化层2\*2窗口和步长s=2，AlexNet步长为2，池化窗口为3\*3

#### GoogleNet
- 22层
- 使用了1\*1的最小卷积层
- 采用了模块化设计的思想

### 9. CIFAR10与VGG13实战

In [ ]:
# 读取data
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.cifar100.load_data()

In [ ]:
# 删除y的一个维度
train_y = tf.squeeze(train_y, axis = 1)
test_y = tf.squeeze(test_y, axis = 1)

# 打印训练集和测试集形状
train_x.shape, train_y.shape, test_x.shape, test_y.shape

In [ ]:
 # 自定义的预处理函数
def preprocess(x, y):
    # 调用此函数时会自动传入 x,y 对象
    x = tf.cast(x, dtype=tf.float32) / 255. # 标准化到 0~1
    y = tf.cast(y, dtype=tf.int32) # 转成整形张量
    y = tf.one_hot(y, depth=10) # one-hot 编码
    # 返回的 x,y 将替换传入的 x,y 参数，从而实现数据的预处理功能
    return x,y

In [ ]:
# 构建训练集对象
# from_tensor_slices可以让每个样本的特征与标签一一对应
train_db = tf.data.Dataset.from_tensor_slices((train_x, train_y))

# map函数调用preprocess，进行预处理，batch是分批，每批128个样本
train_db = train_db.shuffle(1000).map(preprocess).batch(128)

In [ ]:
# 构建测试集对象
test_db = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_db = test_db.map(preprocess).batch(128)

In [ ]:
# 从训练集中采样一个Batch，并观察
sample = next(iter(train_db))
print('sample:', sample[0].shape, sample[1].shape,
tf.reduce_min(sample[0]), tf.reduce_max(sample[0]))

In [ ]:
# VGG13应当有13层，这里只是示例
CNN = Sequential([
    layers.Conv2D(1, kernel_size = [3, 3], padding = 'SAME', activation = 'relu'),
    layers.Conv2D(1, kernel_size = [3, 3], padding = 'SAME', activation = 'relu'),
    layers.MaxPool2D(pool_size = [2, 2], strides = 2, padding = 'SAME'),
    layers.Flatten(),
    layers.Dense(256, activation = 'relu'),
    layers.Dense(128, activation = 'relu'),
    layers.Dense(10, activation = 'softmax')
])

In [ ]:
CNN.build(input_shape = [4, 32, 32, 3])
CNN.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])

In [ ]:
CNN.fit(train_db, epochs = 10)

### 10. 卷积层变种
#### 空洞卷积
普通的卷积层为了减少网络的参数量，卷积核的设计通常选择较小的 1x1,3x3 感受野大小。小卷积核使得网络提取特征时的感受野区域有限，但是增大感受野的区域又会增加网络的参数量和计算代价，因此需要权衡设计。

空洞卷积(Dilated/Atrous Convolution)的提出较好地解决这个问题，空洞卷积在普通卷积的感受野上增加一个 dilation rate 参数，用于控制感受野区域的采样步长。

当感受野的采样步长 dilation rate 为 1 时，每个感受野采样点之间的距离为 1，此时的空洞卷积退化为普通的卷积； 但 dilation rate 为 2 时，感受野每 2 个单元采样一个点，每个采样格子之间的距离为 2；同样的方法，dilation rate 为 3， 采样步长为 3。 尽管 dilation rate 的增大会使得感受野区域增大，但是实际参与运算的点数仍然保持不变。

空洞卷积在不增加网络参数的条件下，提供了更大的感受野窗口。 但是在使用空洞卷积设置网络模型时，需要精心设计 dilation rate 参数来避免出现网格效应， 同时较大的dilation rate 参数并不利于小物体的检测、语义分割等任务。

In [ ]:
# 空洞卷积层
x = tf.random.normal([1, 7, 7, 1])

# dilation_rate
layer = tf.keras.layers.Conv2D(1, kernel_size = 3, strides = 1, dilation_rate = 2)
out = layer(x)
out.shape

#### 转置卷积
转置卷积通过在输入之间填充大量的 padding 来实现输出高宽大于输入高宽的效果，从而实现向上采样的目的。

转置卷积的目的：
- CNN可视化，通过反卷积将卷积得到的feature map还原到像素空间，来观察feature map对哪些pattern相应最大，即可视化哪些特征是卷积操作提取出来的；
- FCN全卷积网络中，由于要对图像进行像素级的分割，需要将图像尺寸还原到原来的大小，类似upsampling的操作，所以需要采用反卷积；
- GAN对抗式生成网络中，由于需要从输入图像到生成图像，自然需要将提取的特征图还原到和原图同样尺寸的大小，即也需要反卷积操作。

In [ ]:
# 转置卷积
x = tf.random.normal([1, 2, 2, 1])
w = tf.constant([[1, 2, 3.], [4, 5, 6], [7, 8, 9]])
w = tf.expand_dims(w, axis = 2)
w = tf.expand_dims(w, axis = 3)
xx = tf.nn.conv2d_transpose(x, w, strides = 2, padding = 'VALID', 
                                          output_shape = [1, 5, 5, 1])
xx.shape

In [ ]:
# 转置卷积层
layer = tf.keras.layers.Conv2DTranspose(1, kernel_size = 3, 
                                        strides = 1, 
                                        padding = 'VALID'
                                       )


#### 分离卷积
普通卷积在对多通道输入进行运算时，卷积核的每个通道与输入的每个通道分别进行卷积运算，得到多通道的特征图，再对应元素相加产生单个卷积核的最终输出。

分离卷积的计算流程则不同，卷积核的每个通道与输入的每个通道进行卷积运算，得到多个通道的中间特征。这个多通道的中间特征张量接下来进行多个 1x1 卷集核的普通卷积运算，得到多个高宽不变的输出，这些输出在通道轴上面进行拼接，从而产生最终的分离卷积层的输出。分离卷积层包含了两步卷积运算，第一步卷积运算是单个卷积核，第二个卷积运算包含了多个卷。

那么采用分离卷积有什么优势呢？一个很明显的优势在于，同样的输入和输出，采用
Separable Convolution的参数量约是普通卷积的三分之一。

比如三通道，三通道3x3的卷积核4个，则其参数量为3x3x3x4=108个。

而分离卷积的第一部分为3x3x3x1=27个，第二部分为1x1x3x4=14个，总共39个。

### 11. 深度残差网络
AlexNet， VGG， GoogLeNet 等网络模型的出现将神经网络的发展带入了几十层的阶段，研究人员发现网络的层数越深，越有可能获得更好的泛化能力。但是当模型加深以后，网络变得越来越难训练，这主要是由于梯度弥散现象造成的。在较深层数的神经网络中间，梯度信息由网络的末层逐层传向网络的首层时， 传递的过程中会出现梯度接近于 0 的现象。 网络层数越深，梯度弥散现象可能会越严重。

那么怎么解决深层神经网络的梯度弥散现象呢？一个很自然的想法是，既然浅层神经网络不容易出现梯度弥散现象，那么可以尝试给深层神经网络添加一种回退到浅层神经网络的机制。当深层神经网络可以轻松地回退到浅层神经网络时，深层神经网络可以获得与浅层神经网络相当的模型性能，而不至于更糟糕。

通过在输入和输出之间添加一条直接连接的 Skip Connection 可以让神经网络具有回退的能力。以 VGG13 深度神经网络为例， 假设观察到 VGG13 模型出现梯度弥散现象，而10 层的网络模型并没有观测到梯度弥散现象，那么可以考虑在最后的两个卷积层添加 SkipConnection，如图 10.62 中所示： 通过这种方式网络模型可以自动选择是否经由这两个卷积层完成特征变换，还是直接跳过这两个卷积层而选择 Skip Connection，亦或结合两个卷积层和 Skip Connection 的输出。

#### ResNet算法
ResNet 通过在卷积层的输入和输出之间添加 Skip Connection 实现层数回退机制，输入𝑥通过两个卷积层，得到特征变换后的输出ℱ(𝑥)，与输入𝑥进行对应元素的相加运算，得到最终输出。

H(𝑥)叫做残差模块。由于被 Skip Connection 包围的卷积神经网络需要学习映射F(𝑥) = H(𝑥) − 𝑥，故称为残差网络。

In [ ]:
# ResBlock实现
from tensorflow.keras import layers
class BasicBlock(layers.Layer):
    def __init__(self, filter_num, stride = 1):
        super(BasicBlock, self).__init__()
        
        # 卷积层1
        self.conv1 = layers.Conv2D(filter_num, (3, 3), 
                                   strides = stride, padding = 'same'
                                  )
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation('relu')
        
        # 卷积层2
        self.conv1 = layers.Conv2D(filter_num, (3, 3), 
                                   strides = stride, padding = 'same'
                                  )
        self.bn1 = layers.BatchNormalization()
        
        # 当F(x)与x形状不同时无法直接相加
        # 需要建立identity(x)卷积层，完成x的形状转换
        if stride != 1:
            self.downsample = Sequential()
            self.downsample.add(layers.Conv2D(filter_num, (1, 1), 
                                              strides = stride))
        else:
            self.downsample = lambda x : x
    
    def call(self, inputs, training = None):
        out = self.conv1(inputs)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        
        # 通过identity()转换
        identity = self.downsample(inputs)
        
        # f(x) + x运算
        output = layers.add([out, identity])
        
        # 通过激活函数
        output = tf.nn.relu(output)
        return output

### 12. DenseNet
DenseNet 将前面所有层的特征图信息通过 Skip Connection 与当前层输出进行聚合，与ResNet 的对应位置相加不同， DenseNet 采用在通道轴 c 维度进行拼接操作， 聚合特征信息。

输入𝑥0通过H1卷积层得到输出𝑥1， 𝑥1与𝑥0在通道轴上进行拼接，得到聚合后的特征张量，送入H2卷积层，得到输出𝑥2，同样的方法， 𝑥2与前面所有层的特征信息: 𝑥1与𝑥0进行聚合，再送入下一层。如此循环，直至最后一层的输出𝑥4和前面所有层的特征信息： {𝑥𝑖}𝑖=0 1 2 3进行聚合得到模块的最终输出。这样一种基于 Skip Connection 稠密连接的模块叫做 Dense Block。

### 13. CIFAR10和ResNet18实战

In [ ]:
from tensorflow.keras import layers, Sequential

In [ ]:
class BasicBlock(layers.Layer):
    
    # 残差模块
    def __init__(self, filter_num, stride=1):
        super(BasicBlock, self).__init__()
        
        # 第一个卷积单元
        self.conv1 = layers.Conv2D(filter_num, (3, 3), strides=stride,
                                   padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation('relu')
        
        # 第二个卷积单元
        self.conv2 = layers.Conv2D(filter_num, (3, 3), strides=1,
                                   padding='same')
        self.bn2 = layers.BatchNormalization()
        
        # 通过 1x1 卷积完成 shape 匹配
        if stride != 1:
            self.downsample = Sequential()
            self.downsample.add(layers.Conv2D(filter_num, (1, 1),
                                strides=stride))
        # shape 匹配，直接短接
        else:
            self.downsample = lambda x:x
    
    def call(self, inputs, training=None):
    
        # [b, h, w, c]，通过第一个卷积单元
        out = self.conv1(inputs)
        out = self.bn1(out)
        out = self.relu(out)

        # 通过第二个卷积单元
        out = self.conv2(out)
        out = self.bn2(out)

        # 通过 identity 模块
        identity = self.downsample(inputs)
        
        # 2 条路径输出直接相加
        output = layers.add([out, identity])
        
        # 激活函数
        output = tf.nn.relu(output) 
        return output
    
    # 可以通过堆叠通道数逐渐增大的 Res Block 来实现高层特征的提取
    # 通过 build_resblock 可以一次完成多个残差模块的新建
    def build_resblock(self, filter_num, blocks, stride=1):
    
        # 辅助函数，堆叠 filter_num 个 BasicBlock
        res_blocks = Sequential()
        
        # 只有第一个 BasicBlock 的步长可能不为 1，实现下采样
        res_blocks.add(BasicBlock(filter_num, stride))
        
        #其他 BasicBlock 步长都为 1
        for _ in range(1, blocks):
            res_blocks.add(BasicBlock(filter_num, stride=1))
        
        return res_blocks

In [ ]:
# 通用的 ResNet 实现类
class ResNet(tf.keras.Model):
    
    # [2, 2, 2, 2]
    def __init__(self, layer_dims, num_classes=10): 
        super(ResNet, self).__init__()
        
        # 根网络，预处理
        self.stem = Sequential([layers.Conv2D(64, (3, 3), strides=(1, 1)),
                                layers.BatchNormalization(),
                                layers.Activation('relu'),
                                layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1),
                                                 padding='same')
        ])
        # 堆叠 4 个 Block，每个 block 包含了多个 BasicBlock,设置步长不一样
        self.layer1 = self.build_resblock(64, layer_dims[0])
        self.layer2 = self.build_resblock(128, layer_dims[1], stride=2)
        self.layer3 = self.build_resblock(256, layer_dims[2], stride=2)
        self.layer4 = self.build_resblock(512, layer_dims[3], stride=2)
        
        # 通过 Pooling 层将高宽降低为 1x1
        self.avgpool = layers.GlobalAveragePooling2D()
        
        # 最后连接一个全连接层分类
        self.fc = layers.Dense(num_classes)
    
        # 可以通过堆叠通道数逐渐增大的 Res Block 来实现高层特征的提取
    # 通过 build_resblock 可以一次完成多个残差模块的新建
    def build_resblock(self, filter_num, blocks, stride=1):
    
        # 辅助函数，堆叠 filter_num 个 BasicBlock
        res_blocks = Sequential()
        
        # 只有第一个 BasicBlock 的步长可能不为 1，实现下采样
        res_blocks.add(BasicBlock(filter_num, stride))
        
        #其他 BasicBlock 步长都为 1
        for _ in range(1, blocks):
            res_blocks.add(BasicBlock(filter_num, stride=1))
        
        return res_blocks
    
    def call(self, inputs, training=None):
        # 通过根网络
        x = self.stem(inputs)
        
        # 一次通过 4 个模块
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        # 通过池化层
        x = self.avgpool(x)
        
        # 通过全连接层
        x = self.fc(x)
        
        return x

In [ ]:
# 通过调整模块内部 BasicBlock 的数量和配置实现不同的 ResNet
def resnet18():
    return ResNet([2, 2, 2, 2])

def resnet34():
    return ResNet([3, 4, 6, 3])

In [ ]:
# 加载数据集
(x,y), (x_test, y_test) = tf.keras.datasets.cifar10.load_data() 

# 删除不必要的维度
y = tf.squeeze(y, axis=1) 
y_test = tf.squeeze(y_test, axis=1)

print(x.shape, y.shape, x_test.shape, y_test.shape)

In [ ]:
# 预处理
def preprocess(x, y):
    
    # 将数据映射到-1~1
    x = 2*tf.cast(x, dtype=tf.float32) / 255. - 1
    
    # 类型转换
    y = tf.cast(y, dtype=tf.int32) 
    return x,y

In [ ]:
# 构建训练集
train_db = tf.data.Dataset.from_tensor_slices((x,y))

# 随机打散，预处理，批量化
train_db = train_db.shuffle(1000).map(preprocess).batch(512)

#构建测试集
test_db = tf.data.Dataset.from_tensor_slices((x_test,y_test)) 

# 随机打散，预处理，批量化
test_db = test_db.map(preprocess).batch(512)

In [ ]:
model = resnet18()

# 训练 epoch
for epoch in range(50): 
    
    for step, (x,y) in enumerate(train_db):
        
        with tf.GradientTape() as tape:
            
            # [b, 32, 32, 3] => [b, 10],前向传播
            logits = model(x)
            
            # [b] => [b, 10],one-hot 编码
            y_onehot = tf.one_hot(y, depth=10)
            
            # 计算交叉熵
            loss = tf.losses.categorical_crossentropy(y_onehot, logits,
                                                      from_logits=True)
            loss = tf.reduce_mean(loss)
            
        # 计算梯度信息
        grads = tape.gradient(loss, model.trainable_variables)
        
        # 更新网络参数
        optimizer.apply_gradients(zip(grads, model.trainable_variables))